In [2]:
import pandas as pd
import matplotlib as plt
import datetime
import typing

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots

%load_ext autoreload
%autoreload 2

In [3]:
data = pd.read_csv('../DATA/processed/dataset.csv', parse_dates=['date'])
data = data.set_index('date')

# Add lag variable

In [5]:
data.columns

Index(['P1', 'P2', 'pressure', 'temperature', 'humidity', 'temp_meteo',
       'pres_meteo', 'hum_meteo', 'wind_direction', 'wind_speed',
       'precipitation', 'prec_amount', 'visibility', 'dew_point_temp'],
      dtype='object')

In [6]:
columns = ['P1', 'P2', 'temp_meteo', 'pres_meteo', 'hum_meteo', 'wind_direction', 'wind_speed', 'prec_amount']

In [7]:
shift_values = [1, 12, 12*2, 12*4, 12*8, 12*24]

In [14]:
for c in columns:
    for s in shift_values:
        data[f'{c}_shift_{s}'] = data[c].shift(s)

# Train test split